# Title Simple Stock Price Prediction Model
Here I will be implementing a simple MLP neural network for stock price prediction.  My GitHub profile is htjames0 and all code and files can be found in the repository. 

In [2]:
import tensorflow as tf
import pandas as pd 
import matplotlib.pyplot as plt

Init Plugin
Init Graph Optimizer
Init Kernel


In [50]:
data = pd.read_csv('AMD.csv')
#features will be Open, High, Low, and Volume
#label will be Close

#dividing the data set into train and test data
train_end = data[data.Date=='2021-01-04'].index[0]
train_data = data.iloc[:train_end,:]
test_data = data.iloc[train_end:,:]

#need to divide training data to account for the prediction days
#i.e. how many days to look back in order to predict the next day
#I plan to expirement with several windows - 15, 30, 45, 90 but 
#starting out with 90
days = 90

#partitioning data so to add a correct label for the number of features
for i in range(days, len(tran_data)):
    #features
    x_train
    #label
    y_train

            Date       Open       High        Low      Close  Adj Close  \
0     2016-01-04   2.770000   2.820000   2.630000   2.770000   2.770000   
1     2016-01-05   2.770000   2.800000   2.640000   2.750000   2.750000   
2     2016-01-06   2.660000   2.710000   2.470000   2.510000   2.510000   
3     2016-01-07   2.430000   2.480000   2.260000   2.280000   2.280000   
4     2016-01-08   2.360000   2.420000   2.100000   2.140000   2.140000   
...          ...        ...        ...        ...        ...        ...   
1254  2020-12-24  91.800003  92.510002  91.309998  91.809998  91.809998   
1255  2020-12-28  92.930000  93.139999  90.820000  91.599998  91.599998   
1256  2020-12-29  91.660004  92.459999  89.430000  90.620003  90.620003   
1257  2020-12-30  90.779999  92.849998  90.190002  92.290001  92.290001   
1258  2020-12-31  92.099998  92.300003  90.870003  91.709999  91.709999   

        Volume  
0     32516800  
1     12972300  
2     23759400  
3     22203500  
4     31822400

In [7]:
def MLPmodel(features, step_size, hidden_neurons=5, act='relu', loss_fxn='mean_squared_error'):
    
    #method inputs
        #features - feature data used to get the number of input layer nodes
        #step-size - for optimizer
        #hidden_neurons - number of neurons in hidden layer, default of five
        #act - activation function, default ReLu function
        #loss_fxn - loss function, default as mean squared error
    
    #defining model
    model = tf.keras.models.Sequential()
   
    #layers
    model.add(tf.keras.layers.Dense(units=hidden_neurons,
                                    input_shape=len(features.columns),
                                    activation='relu')
                                   )
    model.add(tf.keras.layers.Dense(units = 1,
                                    activation=act)
                                   )
    
    #optimizer
    opt = tf.keras.optimizers.Adam(learn_rate=step_size)
    
    #compile - loss fxn MSE
    model.compile(optimizer=opt,
                  loss=loss_fxn,
                  metrics=[tf.keras.metrics.RootMeanSquaredError()])
   
    return model 

In [12]:
#train function
def train_model(model, feature, label, epochs, batch_size):

    #feeding features and labels to model, model iterates epoch number of times
    #using batch_size number of data points per iteration
    history = model.fit(x=feature,
                        y=label,
                        batch_size=batch_size,
                        epochs=epochs)

    #weights and bias
    trained_weight = model.get_weights()[0]
    trained_bias = model.get_bias()[1]

    #historical data of model for each epoch
    epochs = history.epoch
    hist = pd.DateFram(history.history)

    #rmse for each epoch
    rmse = hist["root_mean_squared_error"]

    return trained_weight, trained_bias, epochs, rmse

In [13]:
#predict function
